In [1]:
import tsplib95
import numpy as np
import pandas as pd
import math

### Functions that will be called in the principal code

In [2]:
def fitness(init_pop, n, dimension, upper_b, lower_b, fobj):
    fit1 = np.zeros((n,1))

    for i in range(init_pop, 1):
        fit1[i] = fobj[init_pop[i:]]
    
    [best_fit, idx] = np.min(fit1)
    pos = idx

    return fit1, best_fit, pos

In [3]:
def herding(pop, Vt, fit, n, dimension, acc, time):
    [fit1, idx] = np.sort(fit);
    pop1 = np.zeros((n, dimension))
    Vt1 = np.zeros((n, dimension))
    acc1 = np.zeros((n, dimension))
    time1 = np.zeros((n, dimension))
    
    for i in range(1, n):
        pop1[i, :] = pop[idx[i], :]
        Vt1[i, :] = Vt[idx[i], :]
        acc1[i, :] = acc[i, :]
        time1[i] = time[i]
    
    return pop1, Vt1, acc1, time1

In [4]:
def generate(n, dimension, upper_b, lower_b):
    boundary_no = np.size(upper_b, 2)
    acc = np.zeros((n, dimension))
    
    if boundary_no == 1:
        acc = np.random.random(size = (n, dimension)) * ((upper_b - lower_b) + lower_b)

    if boundary_no > 1:
        for i in range(1, dimension):
            upper_i = upper_b[i]
            lower_i = lower_b[i]
            #Entender a linha de codigo abaixo
            #Positions(:,i)=rand(n,1).*(ub_i-lb_i)+lb_i;

    x = np.random.random(size = (n, dimension))
    M = acc

    return M, x

In [5]:
def update(pop, Vt, time, acc, n, dimension, eye):
    pop1 = np.zeros(n, dimension)
    for i in range(1, n):
        for j in range(1, dimension):
            #Updating the position of dogs
            if (i <= 3):
                pop1[i,j] = Vt[i,j] * time[i] + (1/2) * acc[i,j] * (time[i]^2)
            
            #Updating position of sheep
            if (i > 3):
                if eye == 1:
                    pop1[i,j] = Vt[i,j] * time[i] - (1/2) * acc[i,j] * (time[i]^2)
                else:
                    pop1[i,j] = Vt[i,j] * time[i] + (1/2) * acc[i,j] * (time[i]^2)
            
    return pop1

In [ ]:
def updateV(Vt, n, dimension, acc, time, pop, fit, eye):
    #Choosing left and right dogs
    right_dog = np.random.randint(2,3)
    if (right_dog == 2):
        left_dog = 3
    else:
        left_dog = 2

    acc1 = np.ones((n,dimension))
    time1 = np.ones((n,1))
    Vt1 = Vt
    [r, l] = len(acc)
    acc2 = np.zeros((r, l))
    fit1 = fit[1]

    #Finding Dg value to choose which sheep to gather and which to stalk
    fit2 = (fit[2] + fit[3]) / 2
    f = 0
    tempg = 0
    temps = 0

    #Setting parameters for eyeing
    if eye == 1:
        if fit[right_dog] < fit[left_dog]:
            acc2[left_dog, :] = -1 * acc[left_dog, :]
            f = left_dog
        else:
            acc2[right_dog, :] = -1 * acc[right_dog, :]
            f = right_dog

    for i in range(1, n):
        for j in range(1, dimension):
            #Velocity updation of dogs
            if (i <= 3):
                Vt1[i,j] = math.sqrt(Vt[i,j]^2 + (2 * acc[i,j]) * (pop[i,j]))

            #Velocity updation of sheep
            if (i > 3):
                if eye == 1:
                    Vt1[i,j] = math.sqrt(Vt1[f,j]^2 + (2*acc[f,j]) * pop[i,j])

                else:
                    #Velocity updation of gathered sheep
                    if (fit1 - fit[i] > fit2 - fit[i]):
                        Vt1[i,j] = math.sqrt(Vt1[1,j]^2 + (2*acc[1,j]))        
                
                    #Velocity updation of stalked sheep
                    if (fit1 - fit[i] <= fit2 - fit[i]):
                        Vt1[i,j] = math.sqrt((Vt1[right_dog, j] * math.tan(np.random.randint(1,89))^2) + ((2*acc[right_dog, j]) * pop[right_dog, j]) + (math.sqrt(Vt1[left_dog, j]) * math.tan(np.random.randint(91,179))^2) + (2*acc[left_dog, j]) * pop[left_dog, j])
                        Vt1[i,j] = (Vt1[i,j]) / 2
                        temps[i] = i
    
    #Updation of time and acceleration
    for i in range(1, n):
        s = 0
        for j in range(1, dimension):
            acc[i,j] = abs(Vt1[i,j] - Vt[i,j]) / time[i]
            s += (Vt1[i,j] - Vt[i,j]) / acc[i,j]
        time1[i] = abs(np.mean(s))

    return Vt1, acc1, time1, right_dog, left_dog, tempg, temps

In [6]:
def check(pop, n, dimension, upper_b, lower_b, acc, Vt, time):
    pop1 = pop
    acc1 = acc
    time1 = time
    Vt1 = Vt
    for i in range(1, n):
        for j in range(1, dimension):

            if (pop[i,j] >= upper_b or pop[i,j] <= lower_b or pop[i,j] == 0):
                pop1[i,j] = np.random.random() * (upper_b - lower_b) + lower_b
                acc1[i,j] = np.random.random()
                time1[i] = np.random.random()
    
    for i in range(1, n):
        for j in range(1, dimension):
            if (math.isnan(acc[i,j]) == 1 or acc[i,j] == 0):
                pop1[i,j] = np.random.random() * (upper_b - lower_b) + lower_b
                acc1[i,j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(1, n):
        for j in range(1, dimension):
            if (math.isnan(Vt[i,j]) == 1 or Vt[i,j] == 0):
                pop1[i,j] = np.random.random() * (upper_b - lower_b) + lower_b
                acc1[i,j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(1, n):
        for j in range(1, dimension):
            if (math.isnan(time1[i]) == 1 or time1[i] == 0):
                pop1[i,j] = np.random.random() * (upper_b - lower_b) + lower_b
                acc1[i,j] = np.random.random()
                time1[i] = np.random.random()


    return pop1, acc1, time1, Vt1